In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

### Import dữ liệu sau processed

In [2]:
df = pd.read_csv('Reviews_preprocessed.csv')
df.head(10)

,IDRestaurant,Comment_new,rating_encoded
0,1,chiên sống,1
1,1,ngon,1
2,1,thừa,0
3,1,kêu đói háo_hức thành khóc ròng khóc ròng khóc...,1
4,1,tặng uống,2
5,1,thề lắm tức_giận mừng háo_hức khô chú_ý minion...,0
6,1,thử hơi kén không_ngon_miệng dập tắt nghiện cư...,2
7,1,kết_hợp không_thấy không_cấp,0
8,1,dở ăn_tây tây ngon,1
9,1,cười,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29369 entries, 0 to 29368
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   IDRestaurant    29369 non-null  int64 
 1   Comment_new     29369 non-null  object
 2   rating_encoded  29369 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 688.5+ KB


# Lựa chọn model tốt nhất: KNN, Logistic Regression, Decision Tree, Random Forest, SVM, GaussianNB

In [4]:

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split, cross_val_score
import time

# List of models
models = [   
    GaussianNB(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=50),
    RandomForestClassifier(n_estimators=100),
    SVC(kernel='linear'),
    SVC(kernel='rbf'),
    KNeighborsClassifier(n_neighbors=6),
    KNeighborsClassifier(n_neighbors=12)
]


In [5]:
X = df['Comment_new']
y = df['rating_encoded']

In [6]:
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.3, random_state=42)

In [7]:
# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the training data (handling NaN values internally)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train.astype(str)).toarray()

# Transform the testing data (handling NaN values internally)
X_test_tfidf = tfidf_vectorizer.transform(X_test.astype(str)).toarray()


In [8]:
def get_model_name(model):
    model_name = model.__class__.__name__
    if model_name == 'SVC':
        model_name = model_name + '_' + model.kernel
    elif model_name == 'KNeighborsClassifier':
        model_name = model_name + '_' + str(model.n_neighbors)
    elif model_name == 'RandomForestClassifier':
        model_name = model_name + '_' + str(model.n_estimators)
    return model_name


In [9]:
from sklearn.metrics import accuracy_score

model_selection = pd.DataFrame(columns=['model', 'accuracy', 'time'])

# Loop through the models
for model in models:
    # Get the model name
    model_name = get_model_name(model)
    # Get the current time
    start_time = time.time()
    # Train the model
    model.fit(X_train_tfidf, y_train)
    # Predict the labels for the testing data
    y_pred = model.predict(X_test_tfidf)
    # Calculate the accuracy score
    accuracy = accuracy_score(y_test, y_pred)
    # Get the current time
    end_time = time.time()
    # Calculate the time taken to train the model
    time_taken = end_time - start_time
    # Store the results in the model_selection DataFrame
    model_selection = model_selection._append({
        'model': model_name,
        'accuracy': accuracy,
        'time': time_taken
    }, ignore_index=True)


C:\Users\ttpho\AppData\Local\Temp\ipykernel_13800\3161512827.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  model_selection = model_selection._append({

KeyboardInterrupt



In [1]:
model_selection

NameError: name 'model_selection' is not defined

In [ ]:
# Plot the model selection DataFrame
import matplotlib.pyplot as plt
import seaborn as sns

# Plot the mean accuracy
plt.figure(figsize=(10, 5))
sns.barplot(x='mean_accuracy', y='model', data=model_selection, color='blue')
plt.title('Mean Accuracy')
plt.xlim(0.8, 1)

In [ ]:
1